# Joining Tables with `dfply`

In [1]:
import pandas as pd
from dfply import *

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Example

The following tables were taken from the Wikipedia page for joins and will provide a small but sufficiently complicated example for introducing joins.

In [3]:
dept = pd.read_csv("./data/department.csv")
dept

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [4]:
empl = pd.read_csv("./data/employee.csv")
empl

,DeptID,LastName
0,31.0,Rafferty
1,33.0,Jones
2,33.0,Heisenberg
3,34.0,Robinson
4,34.0,Smith
5,NaN,Williams


In [5]:
dept = pd.read_csv("./data/department.csv")
empl = pd.read_csv("./data/employee.csv")

## Left Outer Join

Use `left_table >> dfply.left_join(right_table, by=common_col_name)`

In [6]:
empl >> left_join(dept, by='DeptID')

,DeptID,LastName,DeptName
0,31.0,Rafferty,Sales
1,33.0,Jones,Engineering
2,33.0,Heisenberg,Engineering
3,34.0,Robinson,Clerical
4,34.0,Smith,Clerical
5,NaN,Williams,NaN


## Rename when the column names are different

In [7]:
empl2 = empl.fillna(0) >> rename(deptIDL = 'DeptID') 
empl2.head(1)

,deptIDL,LastName
0,31.0,Rafferty


In [8]:
empl2 >> left_join(dept, by='DeptID')

KeyError: 'DeptID'

In [9]:
empl2 >> rename(DeptID = 'deptIDL') >> left_join(dept, by='DeptID')

,DeptID,LastName,DeptName
0,31.0,Rafferty,Sales
1,33.0,Jones,Engineering
2,33.0,Heisenberg,Engineering
3,34.0,Robinson,Clerical
4,34.0,Smith,Clerical
5,0.0,Williams,NaN


## Right Outer Join

Use `left_table >> dfply.right_join(right_table, by=common_col_name)`

In [10]:
empl >> right_join(dept, by='DeptID')

,DeptID,LastName,DeptName
0,31.0,Rafferty,Sales
1,33.0,Jones,Engineering
2,33.0,Heisenberg,Engineering
3,34.0,Robinson,Clerical
4,34.0,Smith,Clerical
5,35.0,NaN,Marketing


## Full Outer Join

Use `left_table >> dfply.outer_join(right_table, by=common_col_name)`

In [11]:
empl >> outer_join(dept, by='DeptID')

,DeptID,LastName,DeptName
0,31.0,Rafferty,Sales
1,33.0,Jones,Engineering
2,33.0,Heisenberg,Engineering
3,34.0,Robinson,Clerical
4,34.0,Smith,Clerical
5,NaN,Williams,NaN
6,35.0,NaN,Marketing


## Inner Join

Use `left_table >> dfply.inner_join(right_table, by=common_col_name)`

In [12]:
empl >> inner_join(dept, by='DeptID')

,DeptID,LastName,DeptName
0,31.0,Rafferty,Sales
1,33.0,Jones,Engineering
2,33.0,Heisenberg,Engineering
3,34.0,Robinson,Clerical
4,34.0,Smith,Clerical


## Example 2 - Joining Batting to People

As an example of a joining on more than 1 column, consider the following task.

**Task:** Create a table with the total runs score for each team in 2009.  Include the teams proper name and the name of their home park.

In [13]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/Teams.csv")

In [14]:

batting, teams = [pd.read_csv(f) for f in files]

## Things we need from `Batting.csv`

* Year
* Runs
* Team

In [15]:
batting.head(2)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0


#### Selecting down to the necessary columns

In [16]:
batting_select = (batting
                  >> select(X.teamID,
                            X.yearID,
                            X.R)
                 )
batting_select.head()

,teamID,yearID,R
0,TRO,1871,0
1,RC1,1871,30
2,CL1,1871,28
3,WS3,1871,28
4,RC1,1871,29


In [17]:
batting_select.shape

(104324, 3)

## Things we need from `Teams.csv`

* Year
* Team
* Team Name
* Team Park

In [18]:
teams.head(2)

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1


#### Selecting down to the necessary columns

In [19]:
teams_select = (teams >> select(X.yearID,
                                X.teamID,
                                X.name,
                                X.park))
teams_select.head()

,yearID,teamID,name,park
0,1871,BS1,Boston Red Stockings,South End Grounds I
1,1871,CH1,Chicago White Stockings,Union Base-Ball Grounds
2,1871,CL1,Cleveland Forest Citys,National Association Grounds
3,1871,FW1,Fort Wayne Kekiongas,Hamilton Field
4,1871,NY2,New York Mutuals,Union Grounds (Brooklyn)


## How can we match these columns?

We will need to match on both `teamID` and `yearID` because

1. Team names might change
2. The teams park depends on the year.

## Determining the correct join.

Facts of the task:

* We want batting statistics for all players in 2009
* We want only teams names and parks for teams that played that year.

**Question:** Which join? 
**Answer:** `batting >> left_join(teams)` on both year and 

## How to join on multiple columns

Pass a list of columns names to `by`

In [20]:
df_joined = batting_select >> left_join(teams_select, by=['teamID', 'yearID'])
df_joined.head()

,teamID,yearID,R,name,park
0,TRO,1871,0,Troy Haymakers,Haymakers' Grounds
1,RC1,1871,30,Rockford Forest Citys,Agricultural Society Fair Grounds
2,CL1,1871,28,Cleveland Forest Citys,National Association Grounds
3,WS3,1871,28,Washington Olympics,Olympics Grounds
4,RC1,1871,29,Rockford Forest Citys,Agricultural Society Fair Grounds


#### Filter, group by, and aggregate to get the desired result.

In [21]:
runs_per_team09 = (df_joined
                   >> filter_by(X.yearID == 2009)
                   >> group_by(X.name, X.park)
                   >> summarise(total_runs = X.R.sum()))
runs_per_team09.head()

,park,name,total_runs
0,Chase Field,Arizona Diamondbacks,720
1,Turner Field,Atlanta Braves,735
2,Oriole Park at Camden Yards,Baltimore Orioles,741
3,Fenway Park II,Boston Red Sox,872
4,Wrigley Field,Chicago Cubs,707


In [22]:
runs_per_team09.shape

(30, 3)

## Joins are expensive!

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `inner_join` performs `min(len(A), len(B))` comparisons
* `outer_join` performs `len(A)*len(B)` row comparisons.
* `left_join` and `right_join` are somewhere inbetween (depending on how we treat multiples).

## Outer joins (can) produce LARGE tables.

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `outer_join` might produce `len(A)*len(B)` rows (worst case)
* `left_join` and `right_join` are similar if we don't drop multiples.

#### Last example 

In [23]:
teams_select.shape

(2865, 4)

In [24]:
len(batting_select)*len(teams_select)

298888260

## The joining mantra: Filter/aggregate THEN join

By filtering and/or aggregating before joining, you will

* reduce the amount of work
* protect against really large output

#### Filter and aggregate to total runs per team per year

In [25]:
runs = (batting_select
        >> filter_by(X.yearID == 2009)
        >> group_by(X.teamID)
        >> summarise(total_runs = X.R.sum()))

runs.head()

,teamID,total_runs
0,ARI,720
1,ATL,735
2,BAL,741
3,BOS,872
4,CHA,724


#### Filter down to 2009 teams

In [26]:
teams09 = (teams_select
           >> filter_by(X.yearID == 2009)
           >> drop(X.yearID)
          )
teams09.head()

,teamID,name,park
2595,ARI,Arizona Diamondbacks,Chase Field
2596,ATL,Atlanta Braves,Turner Field
2597,BAL,Baltimore Orioles,Oriole Park at Camden Yards
2598,BOS,Boston Red Sox,Fenway Park II
2599,CHA,Chicago White Sox,U.S. Cellular Field


In [27]:
len(runs),len(teams09)

(30, 30)

In [28]:
len(runs)*len(teams09)

900

## Joining the smaller tables

**Note:** Since we filtered down to 2009, we don't need to join on `yearID`

In [29]:
runs_per_team09_new = (runs 
                       >> left_join(teams09, by='teamID')
                       >> drop(X.teamID))
runs_per_team09_new.head()

,total_runs,name,park
0,720,Arizona Diamondbacks,Chase Field
1,735,Atlanta Braves,Turner Field
2,741,Baltimore Orioles,Oriole Park at Camden Yards
3,872,Boston Red Sox,Fenway Park II
4,724,Chicago White Sox,U.S. Cellular Field


In [30]:
runs_per_team09_new.shape

(30, 3)

## Don't worry about speed until it matters!

* I don't emphasize speed most of the time.
* In this case, either approach was nearly instant.
* Still ... `join`s are *so dangerous* that I make an exception here!

## <font color="red"> Exercise 1 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, use the join methods presented above.

In [31]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/People.csv",
         "./data/baseball/core/Teams.csv")

In [80]:
batting, people, teams = [pd.read_csv(f) for f in files]


In [104]:
batting_reduced = (batting
                   >> select(X.playerID, X.teamID, X.HR, X.yearID)
                   >> filter_by(X.HR >= 50))
batting_reduced >> head


,playerID,teamID,HR,yearID
18532,ruthba01,NYA,54,1920
19052,ruthba01,NYA,59,1921
22269,ruthba01,NYA,60,1927
22799,ruthba01,NYA,54,1928
23978,wilsoha01,CHN,56,1930


In [82]:
people_reduced = (people
                  >> select(X.playerID, X.nameGiven ))
people_reduced >> head

,playerID,nameGiven
0,aardsda01,David Allan
1,aaronha01,Henry Louis
2,aaronto01,Tommie Lee
3,aasedo01,Donald William
4,abadan01,Fausto Andres


In [100]:
teams_reduced = (teams
                 >> select(X.teamID, X.name, X.yearID))
teams_reduced.head()

,teamID,name,yearID
0,BS1,Boston Red Stockings,1871
1,CH1,Chicago White Stockings,1871
2,CL1,Cleveland Forest Citys,1871
3,FW1,Fort Wayne Kekiongas,1871
4,NY2,New York Mutuals,1871


In [84]:
# We should have 44 people in the final file because there are 44 people who hit over 50 hrs
batting_reduced.shape

(44, 3)

In [106]:
HR_over50 = (batting_reduced 
             >> left_join(people_reduced, by='playerID')
             >> left_join(teams_reduced, by=['teamID', 'yearID']))
HR_over50

,playerID,teamID,HR,yearID,nameGiven,name
0,ruthba01,NYA,54,1920,George Herman,New York Yankees
1,ruthba01,NYA,59,1921,George Herman,New York Yankees
2,ruthba01,NYA,60,1927,George Herman,New York Yankees
3,ruthba01,NYA,54,1928,George Herman,New York Yankees
4,wilsoha01,CHN,56,1930,Lewis Robert,Chicago Cubs
5,foxxji01,PHA,58,1932,James Emory,Philadelphia Athletics
6,foxxji01,BOS,50,1938,James Emory,Boston Red Sox
7,greenha01,DET,58,1938,Henry Benjamin,Detroit Tigers
8,kinerra01,PIT,51,1947,Ralph McPherran,Pittsburgh Pirates
9,mizejo01,NY1,51,1947,John Robert,New York Giants


## Up Next

Stuff